## Import libraries

In [ ]:
import numpy as np
import segyio
import matplotlib
import matplotlib.pyplot as plt
import itertools

In [ ]:
# Define the path to seismic file
filename = '/Users/ruslan/Downloads/Ichthys 3D seismic for fault competition.segy'

## View SEGY header

In [ ]:
f = segyio.open(filename, ignore_geometry = True)
segyio.tools.wrap(f.text[0])
print(segyio.tools.wrap(f.text[0]))

## Read SEGY into RAM

In [ ]:
with segyio.open(filename) as segyfile:
    seis_data = segyio.tools.cube(filename)
    xlines = segyfile.xlines
    ilines = segyfile.ilines
    samples = segyfile.samples

## Plot vertical slice with Matplotlib

In [ ]:
# define the range for visualization
vm = np.percentile(seis_data, 95)
print(vm)

In [ ]:
extent = [xlines[0], xlines[-1], samples[-1], samples[0]]
plt.imshow(seis_data[100,:,:].T, vmin=-vm, vmax=vm, cmap='gray', aspect='auto')

plt.xlabel('XLINE')
plt.ylabel('TIME')
plt.title('Ichthys 3D - INL 100')
plt.show()

## Plot time slice with Matplotlib

In [ ]:
# Central time slice

tmid = len(samples)//2

extent = [xlines[0], xlines[-1], ilines[0], ilines[-1]]

plt.imshow(seis_data[:,:,tmid], vmin=-vm, vmax=vm, cmap='gray', origin='lower', aspect='auto', extent=extent)

plt.xlabel('XLINE')
plt.ylabel('INLINE')
plt.title('Ichthys 3D - Time Slice '+ str(samples[tmid])+'ms')
plt.show()

###  Plotting with HOLOVIZ

In [ ]:
import hvplot.xarray
import panel as pn
import xarray as xr
from holoviews import opts
import holoviews as hv
hv.extension('bokeh')

opts.defaults(
    opts.Image(
        width=900, height=700), tools=["layers_control"])

### Interactive functions

In [ ]:
def plot_inl(inl, cmap):
    """
    Plot a single inline using hvplot
    """
    idx = inl
    da = xr.DataArray(seis_data[idx,:,:].T)   
    p = da.hvplot.image( clim=(-vm, vm),extent = [xlines[0], xlines[-1], samples[-1], samples[0]], cmap=cmap, flip_yaxis=True, xlabel='Xline_idx', ylabel='Time_idx', title='ILine '+ str(ilines[inl])) 
    return p
def plot_xln(xln, cmap):
    """
    Plot a single xline using hvplot
    """
    idx = xln
    da = xr.DataArray(seis_data[:,idx,:].T)    
    p = da.hvplot.image(clim=(-vm, vm), cmap=cmap, flip_yaxis=True, xlabel='Iline_idx', ylabel='Time_idx', title='XLine '+ str(xlines[xln])) 
    return p

def plot_top(t, cmap):
    """
    Plot a single time slice using hvplot
    """
    idx = t
    da = xr.DataArray(seis_data[:,:,idx])    
    p = da.hvplot.image(clim=(-vm, vm), cmap=cmap, xlabel='Xline_idx', ylabel='Iline_idx', title='Time '+str(samples[t])+'ms', extent = [xlines[0], xlines[-1], ilines[0], ilines[-1]]) 
    return p

iline_old = 0
xline_old = 0
t_old = 0
def plot_seis_slice(inline_indx, xline_indx, time_indx, cmap):
    global iline_old, xline_old, t_old
    if np.abs(iline_old - inline_indx) > 0:
        iline_old = inline_indx
        return plot_inl(inline_indx, cmap)
    elif np.abs(xline_old - xline_indx) > 0:
        xline_old = xline_indx
        return plot_xln(xline_indx, cmap)
    elif np.abs(t_old - time_indx) > 0:
        t_old = time_indx
        return plot_top(time_indx, cmap)

### Layout

In [ ]:
pallet_select = pn.widgets.Select(name='Pallet', options=['gray', 'seismic', 'bwr'])  
inline_slider = pn.widgets.IntSlider(name='Inline_indx', start=0, end=len(xlines))
xline_slider = pn.widgets.IntSlider(name='Xline_indx', start=0, end=len(xlines))
tline_slider = pn.widgets.IntSlider(name='Time_indx', start=0, end=len(samples))
col = pn.Column(pn.Row (pallet_select), pn.Row (inline_slider, xline_slider, tline_slider))
layout_seis = pn.interact(plot_seis_slice, inline_indx = inline_slider, xline_indx=xline_slider, time_indx = tline_slider, cmap=pallet_select)
pn.Column(col,  layout_seis[1])